# Introduction to GPT-2
by Eugenio Tufino, University of Padova, 9 December, 2024

[Link to Google Colab](https://colab.research.google.com/drive/1MjpwPIb9uQFNLegfbfbXbG2LrgFd-nam?usp=sharing)

In this notebook, we present GPT-2, a simplified model compared to the state-of-the-art generative models such as  GPT-4 or OpenAI o1, the model just released. GPT-2 is based on the **Generative Pre-trained Transformer** architecture and was released by OpenAI in **2019**. It uses a relatively simple mechanism to generate text, predicting the next word in a sequence based on the given context.

Therefore unlike Word2Vec, where as we have seen word vectors are static (the same vector represents a word regardless of its usage), GPT-2 **dynamically adjusts the word representation** based on its context. This allows GPT-2 to better capture the meaning of words in different situations, resulting in more coherent and contextually accurate text generation.

We will explore this model to understand, in a simplified way, how this kind of generative AI works, focusing on its word generation process and how prompts affect its output.

---


GPT stands for Generative Pre-trained Transformer, highlighting its key features:
- Generative: The model generates coherent and meaningful text.
- Pre-trained: It is trained on large datasets before being fine-tuned for specific tasks.
- Transformer: It uses the Transformer architecture for efficient and context-aware text generation.


Further reading:
1. https://en.wikipedia.org/wiki/GPT-2
2. https://huggingface.co/docs/transformers/en/model_doc/gpt2
3. The Transformer architecture was introduced here: Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., Kaiser, Ł., & Polosukhin, I. (2017). "Attention Is All You Need." Advances in Neural Information Processing Systems
4.. https://writings.stephenwolfram.com/2023/02/what-is-chatgpt-doing-and-why-does-it-work/





### Import the libraries for using GPT-2


In [2]:
!pip install transformers
import numpy as np
import matplotlib.pyplot as plt
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

In [3]:
# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"  # This is the smallest size. You can also use 'gpt2-medium', 'gpt2-large'.

# The tokenizer processes input text into tokens and converts tokens back to text

tokenizer = GPT2Tokenizer.from_pretrained(model_name) # Load the pre-trained tokenizer for GPT-2
# The tokenizer processes input text into tokens and converts tokens back to text
#Load the pre-trained GPT-2 language model
# The model generates text by predicting the next token in a sequence
model = GPT2LMHeadModel.from_pretrained(model_name)

print("GPT-2 model and tokenizer loaded successfully!")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT-2 model and tokenizer loaded successfully!


### What does a 'tokenizer' mean, and what is a token?
Check this website (for the latest version of GPT): [Link](https://platform.openai.com/tokenizer)

Try to tokenize the following text:

In [4]:
# Text to tokenize
text = "What is AI?"
# Convert text into tokens
tokens = tokenizer.encode(text, return_tensors="pt")
# Output the tokens
print(tokens)  #print the token Ids

tensor([[2061,  318, 9552,   30]])


### Info about the model GPT-2



In [5]:
# Check vocabulary size and vector size from the model's configuration
vocab_size = model.config.vocab_size
vector_size = model.config.n_embd

print("Vocabulary size:", vocab_size)
print("Vector size (number of dimensions):", vector_size)


Vocabulary size: 50257
Vector size (number of dimensions): 768


## Generating text with GPT-2

In [6]:
# Define a physics-related prompt
prompt = "In physics, energy is the ability to"
# Tokenize the prompt
input_ids = tokenizer.encode(prompt, return_tensors='pt')

In [7]:

# Create an attention mask
attention_mask = torch.ones_like(input_ids)

output = model.generate(
    input_ids,
    attention_mask=attention_mask,  # Specify the attention mask
    max_length=100,#Maximum length of the generated sequence
    num_return_sequences=1,# Number of different sequences to generate
    no_repeat_ngram_size=2,  # # Avoids repeated bigrams, see Annex A
    temperature=0.7,
    top_k=50, #Limits sampling to the top 50 tokens by probability, reducing unlikely choices
    top_p=0.95,# Enables nucleus sampling, selecting tokens with cumulative probability ≤ 0.95
    do_sample=True,  # Enable sampling
    pad_token_id=tokenizer.eos_token_id,  # Set pad token
)
# Decode and print the generated text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:")
print(generated_text)


Generated Text:
In physics, energy is the ability to hold the object in its orbit around a point, as in the case of the sun. The sun's orbit moves in a straight line from point A to point B, and the gravitational field of a star (which is what makes stars move in an orbit) moves outwards from the center of B into the middle of A, or in other words, to the point where the star is at a position in which it is not in orbit. It's a


In a previous session I obtained the following text:
> In physics, energy is the ability to move matter in the universe. This is why the earth is flat. But the way the world works is different.The earth orbits the sun, which is a reflection of the moon, and then it turns around and turns the other way around. The solar system's rotation slows down when the Earth spins, so the solar wind is pulled in two directions. It takes a lot of energy to turn the two sides of a circle around each other...



## Questions
1. How coherent is the generated text with the given prompt?
2. Does the model truly understand the concepts, or is it merely generating plausible sentences?
3. How does its behavior differ from Word2Vec?


GPT-2 generates text dynamically by predicting the next word in a sequence based on its context (through the Transformer architecture). It can produce  produce nonsensical or inaccurate content.


### Parameter you can vary to improve the generated text:
**temperature:** Controls the randomness of text generation. A lower value produces more predictable text, while a higher value increases creativity.
**max_length**: Adjusts the length of the generated text.
**top_k and top_p:** Regulate the selection of probable words at each step.

(see also Annex A below)

**Exercise 1** Adjust the temperature. Experiment with different calues of temperature to improve the quality of GPT-2’s generated output. Observe how it influences the behavior of the model.

 Which temperature value produces the most meaningful text for your physics context?

### We can read the probabilities of the next word after the given prompt
prompt = "Newton's laws state that every object in motion"

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import torch.nn.functional as F

# Load the model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Define a prompt
prompt = "Newton's laws state that every object in motion must"

# Encode the prompt
input_ids = tokenizer.encode(prompt, return_tensors='pt')

# Forward pass through the model to get logits
outputs = model(input_ids)
logits = outputs.logits  # Shape: (batch_size, sequence_length, vocab_size)

# Get the logits for the last token in the input
last_token_logits = logits[0, -1, :]  # Shape: (vocab_size,)

# Set the temperature
temperature = 0.7  # Lower = less random, higher = more random

# Apply temperature to logits
adjusted_logits = last_token_logits / temperature

# Convert adjusted logits to probabilities
probs = F.softmax(adjusted_logits, dim=-1)

# Get the top 10 words with their probabilities
top_k = 10
top_k_probs, top_k_indices = torch.topk(probs, top_k)

# Decode and print the top words with their probabilities
for i in range(top_k):
    word = tokenizer.decode(top_k_indices[i].item())
    prob = top_k_probs[i].item()
    print(f"{word}: {prob:.4f}")



 be: 0.8125
 have: 0.1467
 pass: 0.0065
 conform: 0.0038
 move: 0.0028
 remain: 0.0019
 fall: 0.0015
 first: 0.0014
 ": 0.0014
 bear: 0.0014


**Exercise 2:** Try to change the "temperature" parameter and see if there are any differences

**Exercise 3:** You can continue the construction of the sentence, word by word...

### Iterating the process
Iterate the process changing only the temperature, the max_words and top_k parameters.

In [ ]:

input_ids = tokenizer.encode(prompt, return_tensors='pt')

# Set parameters
max_words = 100  # Maximum number of words to generate
temperature = 0.8
top_k = 50

# Generate word by word
for _ in range(max_words):
    # Forward pass to get logits
    outputs = model(input_ids)
    logits = outputs.logits  # Shape: (batch_size, sequence_length, vocab_size)

    # Get the logits for the last token
    last_token_logits = logits[0, -1, :]

    # Apply temperature
    adjusted_logits = last_token_logits / temperature

    # Convert logits to probabilities
    probs = F.softmax(adjusted_logits, dim=-1)

    # Sample the next token
    next_token_id = torch.multinomial(probs, num_samples=1)

    # Append the token to the input_ids
    input_ids = torch.cat([input_ids, next_token_id.unsqueeze(0)], dim=1)

    # Decode the current output and print
    generated_text = tokenizer.decode(input_ids[0])
    print(generated_text)

# Final generated paragraph
print("\nFinal Generated Paragraph:")
print(tokenizer.decode(input_ids[0]))


Newton's laws state that every object in motion must be
Newton's laws state that every object in motion must be definite
Newton's laws state that every object in motion must be definite.
Newton's laws state that every object in motion must be definite. The
Newton's laws state that every object in motion must be definite. The law
Newton's laws state that every object in motion must be definite. The law required
Newton's laws state that every object in motion must be definite. The law required that
Newton's laws state that every object in motion must be definite. The law required that Bra
Newton's laws state that every object in motion must be definite. The law required that Bravery
Newton's laws state that every object in motion must be definite. The law required that Bravery and
Newton's laws state that every object in motion must be definite. The law required that Bravery and Religion
Newton's laws state that every object in motion must be definite. The law required that Bravery and R

**Exercise: Try to improve the overall output with the following measures**



1. Explore different prompts: You can test different prompts, specific to physics concepts, to observe how the behaviour of the model changes.
2. Change Temperature: You can change the temperature parameter to explore the effect of randomness on generation.
3. You can generalize the code to consider the parameter no_repeat_ngram_size=3  to prevent repetitions of blocks of text.

## Annex A  Parameters for text generation

### **1. `no_repeat_ngram_size`**

What is an n-gram?

An n-gram is a sequence of n consecutive tokens (words or subwords) in text. For example:

    1-gram (Unigram): ["The", "cat", "is", "on", "the", "mat"]
    2-gram (Bigram): ["The cat", "cat is", "is on", "on the", "the mat"]
    
- **Definition**: Prevents the repetition of n-grams (sequences of `n` consecutive tokens) in the generated text.
- **Purpose**: Ensures variety in the output by disallowing repetitive phrases.
- **Example**:
  - With `no_repeat_ngram_size=2`: The bigram "the cat" cannot appear more than once in the text.
  - Without it: Text may loop or repeat, like "The cat is on the mat. The cat is on the mat."



### **2. `temperature`**
- **Definition**: Controls the randomness of token selection during generation.
- **Purpose**: Adjusts the "creativity" of the output.
- **Behavior**:
  - Lower values (e.g., `0.5`): Predictable and conservative text.
  - Higher values (e.g., `1.5`): More creative but can lead to incoherence.



### **3. `top_k`**
- **Definition**: Limits sampling to the top-k most probable tokens.
- **Purpose**: Ensures the model selects tokens from a fixed number of high-probability options.
- **Behavior**:
  - Lower values (e.g., `10`): More focused and deterministic text.
  - Higher values (e.g., `50`): Allows greater diversity but risks introducing randomness.


### **4. `top_p` (Nucleus Sampling)**
- **Definition**: Chooses tokens whose cumulative probability is ≤ `top_p`.
- **Purpose**: Dynamically adjusts the diversity of token selection.
- **Behavior**:
  - Lower values (e.g., `0.8`): Limits to the most likely tokens, producing focused text.
  - Higher values (e.g., `0.95`): Includes more tokens, allowing for greater creativity.

### **5. `max_length`**
- **Definition**: Sets the maximum number of tokens in the generated sequence, including the input.
- **Purpose**: Controls how long the generated text will be.
- **Behavior**:
  - Short sequences (e.g., `50`): Output is concise but might be incomplete.
  - Long sequences (e.g., `200`): Risk of losing coherence or repeating patterns.


### **6. `pad_token_id`**
- **Definition**: Specifies the token used for padding sequences to a uniform length.
- **Purpose**: Ensures the model handles sequences with padding properly during generation.
- **Common Value**: Set to the End-of-Sequence (EOS) token in GPT-2 (`tokenizer.eos_token_id`).


### **7. `attention_mask`**
- **Definition**: A binary mask indicating which tokens in the input are valid (1) and which are padding (0).
- **Purpose**: Ensures the model only attends to meaningful tokens during processing.
- **Behavior**:
  - Without it: The model may produce unreliable results, especially with padded sequences.
